# [LVV-T2214] - MTMount_Elevation_Changes_with_MTAOS_Aberrations

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2214]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2214

## Requirements

This notebooks require some extra repositories to be installed locally so it can grab some constants and some look-up tables.  
Here is a list of which repos are these:

* [lsst-ts/ts_cRIOpy]

[lsst-ts/ts_cRIOpy]: https://github.com/lsst-ts/ts_cRIOpy 
[lsst-ts/ts_m1m3support]: https://github.com/lsst-ts/ts_m1m3support

To install them, open a terminal and run the following commands:

```
git clone https://github.com/lsst-ts/ts_cRIOpy $HOME/notebooks/lsst-ts/ts_cRIOpy
git clone https://github.com/lsst-ts/ts_m1m3support.git $HOME/notebooks/lsst-ts/ts_m1m3support
git clone https://github.com/lsst-ts/ts_config_mttcs $HOME/notebooks/lsst-ts/ts_config_mttcs
git clone https://github.com/lsst-sitcom/M2_FEA $HOME/notebooks/lsst-sitcom/M2_FEA
```

And add these lines to your `$HOME/notebooks/.user_setup` file: 

```
setup -j notebooks_vandv -r ~/notebooks/lsst-sitcom/notebooks_vandv
setup -j ts_cRIOpy -r $HOME/notebooks/lsst-ts/ts_cRIOpy
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

---
## Setup Notebook for Test

- Import all libraries
- Get the remotes ready

In [ ]:
test_execution = "LVV-EXXXX"

In [ ]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

## Helper Functions

In [ ]:
async def get_data_from_efd(location, start, end):
    """
    Retrieves data relevant for analysis.
    
    Parameters
    ----------
    location : str
        In which Test Stand are you running this test?
    start : str or datetime
        Start of the time-window.
    end : str or datetime 
        End of the time-window.
    """
    if location == "summit":
        client = EfdClient("summit_efd")
    elif location == "tucson":
        client = EfdClient("tucson_teststand_efd")
    else:
        raise ValueError(
            "Location does not match any valid options {summit|tucson}"
        )
    
    _df_mount_el = await client.select_time_series(
        "lsst.sal.MTMount.elevation", 
        fields="actualPosition", 
        start=start.utc, 
        end=end.utc,
    )
    
    _df_m1m3_101 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        fields="zForce101",
        start=start.utc, 
        end=end.utc,
    )
        
    # M2 B1?
    _df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=[
            "applied0",
            "lutGravity0",
        ],
        start=start.utc, 
        end=end.utc,
    )
    
    # CamHex Y position
    _df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=1,
        start=start.utc, 
        end=end.utc,
    )
    
    # M2Hex Y position
    _df_m2hex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=2,
        start=start.utc, 
        end=end.utc,
    )

    # Rename columns
    _df_mount_el.rename(columns={"actualPosition": "mount_el"}, inplace=True)
    _df_m1m3_101.rename(columns={"zForce101": "m1m3_z101"}, inplace=True)
    _df_m2.rename(columns={"applied0": "m2b1_applied", "lutGravity0": "m2b1_gravLut"}, inplace=True)
    _df_camhex.rename(columns={"position1": "camhex_y"}, inplace=True)
    _df_m2hex.rename(columns={"position1": "m2hex_y"}, inplace=True)

    
    # Join dataframes
    _df = _df_mount_el
    
    _df = pd.merge(
        _df, 
        _df_m1m3_101, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
         
    _df = pd.merge(
        _df, 
        _df_m2, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    _df = pd.merge(
        _df, 
        _df_camhex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )

    _df = pd.merge(
        _df, 
        _df_m2hex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    return _df

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount
- [ ] MTRotator
- [ ] MTPtg

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

---
## Check elevation motion is smooth and no abrupt changes occur (Otherwise it faults M1M3) 

Do the following slew sequence, watch chronograph to make sure there is no abrupt change in elevation angle (otherwise it faults M1M3).

Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

Move to zenith at the end (so that we can start m1m3 with LUT in mount telemetry mode)

In [ ]:
# Find targets between 86.5 and 82 deg elevation. 
target_1 = mtcs.radec_from_azel(az=230.5, el=85.4)
target_2 = mtcs.radec_from_azel(az=231, el=84.4)
target_3 = mtcs.radec_from_azel(az=230.5, el=83.4)
target_4 = mtcs.radec_from_azel(az=231, el=82.4)

print(f"Target 1: {target_1}\n"
      f"Target 2: {target_2}\n"
      f"Target 3: {target_3}\n"
      f"Target 4: {target_4}\n")

In [ ]:
# Set this to True when you actually want to run this test
if True:
    t_start = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_start.format = "isot"
    print(f"Gathering data - Elevation motion test - Start time: {t_start.utc}")

    await asyncio.sleep(39.)

    # First target
    print('Slewing to first source')
    await mtcs.slew_icrs(ra=target_1.ra, dec=target_1.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Second target
    print('Slewing to second source')
    await mtcs.slew_icrs(ra=target_2.ra, dec=target_2.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Third target
    print('Slewing to third source')
    await mtcs.slew_icrs(ra=target_3.ra, dec=target_3.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Fourth target 
    print('Slewing to fourth source') 
    await mtcs.slew_icrs(ra=target_4.ra, dec=target_4.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Stop tracking
    await mtcs.stop_tracking()
    
    t_end = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_end.format = "isot"
    print(f"Gathering data - Elevation motion test - End time: {t_end.utc}")
    

In [ ]:
# Plot mount elevation as a function of time
# Query telemetry
dfm_elevation = await client.select_time_series('lsst.sal.MTMount.elevation', 'actualPosition', start_time, end_time)

In [ ]:
# Plot 
fig, axs = plt.subplots(figsize=(10, 10), sharex=True)

axs[0].plot(dfm_elevation["actualPosition"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_mount_el_vs_time.png")
plt.show()

In [ ]:
# Move to zenith
await mtcs.point_azel(az=230, el=90)

---
## Get M1M3 Ready

- Raise the mirror,
- Turn on Balance Forces,
- Clear forces

Need to have M1M3 LUT use mount telemetry.

In [ ]:
# If M1M3 is not raised yet, use this command to raise it.
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

In [ ]:
# M1M3 LUT use mount telemetry
# todo: how to do that? 

## Get M2 Ready
- Turn on Force Balance system
- Clear forces

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

In [ ]:
# Need to have M2 LUT use mount telemetry
# todo: how to do that?

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Gather Data - Without Aberrations
Do 4 slews in sequence. For each slew, track for 39s, simulating a visit.
Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [ ]:
# Find new targets
target_1 = mtcs.radec_from_azel(az=120.5, el=85.4)
target_2 = mtcs.radec_from_azel(az=120.5, el=85)
target_3 = mtcs.radec_from_azel(az=120.5, el=84)
target_4 = mtcs.radec_from_azel(az=120.5, el=82.5)

print(f"Target 1: {target_1}\n"
      f"Target 2: {target_2}\n"
      f"Target 3: {target_3}\n"
      f"Target 4: {target_4}\n")

In [ ]:
# Set this to True when you actually want to run this test
if True:
    t_start = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_start.format = "isot"
    print(f"Gathering data - without Aberrations - Start time: {t_start.utc}")

    await asyncio.sleep(39.)

    # First target
    await mtcs.slew_icrs(ra=target_1.ra, dec=target_1.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    # Second target
    await mtcs.slew_icrs(ra=target_2.ra, dec=target_2.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    # Third target
    await mtcs.slew_icrs(ra=target_3.ra, dec=target_3.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    # Fourth target 
    await mtcs.slew_icrs(ra=target_4.ra, dec=target_4.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Stop tracking
    await mtcs.stop_tracking()
    
    t_end = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_end.format = "isot"
    print(f"Gathering data - without Aberrations - End time: {t_end.utc}")

## Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
# Query telemetry
df = await get_data_from_efd(
    exec_info.loc, 
    t_start, 
    t_end)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df["m1m3_z101"].dropna(), "C0o-", label="z")
axs[1].set_ylabel("M1M3 Forces\n[--]")

axs[2].plot(df["m2b1_applied"].dropna(), "C1^-", label="applied")
axs[2].plot(df["m2b1_gravLut"].dropna(), "C2v-", label="Gravity LUT")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_without_aberr.png")
plt.show()

## Gather Data - With Aberrations
Do 4 slews in sequence.
For each slew,

2 seconds before the slew starts, use MTAOS to add some aberrations to the system. The aberrations should correspond to 1um of z4 (focus), z5(astigmatism), z7 (y-coma), and z9 (trefoil), respectively track for 39s, simulating a visit.

Note: the 4 slews need to correspond to elevation angle between 86.5 deg and 82 deg. And, additional margin is needed due to tracking.

In [ ]:
# Find new targets
target_1 = mtcs.radec_from_azel(az=120.5, el=85.4)
target_2 = mtcs.radec_from_azel(az=120.5, el=85)
target_3 = mtcs.radec_from_azel(az=120.5, el=84)
target_4 = mtcs.radec_from_azel(az=120.5, el=82.5)

print(f"Target 1: {target_1}\n"
      f"Target 2: {target_2}\n"
      f"Target 3: {target_3}\n"
      f"Target 4: {target_4}\n")

In [ ]:
# Set this to True when you actually want to run this test
if True:
    t_start = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_start.format = "isot"
    print(f"Gathering data - with Aberrations - Start time: {t_start.utc}")

    await asyncio.sleep(39.)

    # First target   # 1 um of z4 (Focus)
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    wavefront_errors = np.zeros(19)
    wavefront_errors[0] = 1.0 
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    # Wait 2 s and slew to first target
    await asyncio.sleep(2.)
    await mtcs.slew_icrs(ra=target_1.ra, dec=target_1.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)

    # Second target  # 1 um of z5 (Astigmatism)
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    wavefront_errors = np.zeros(19)
    wavefront_errors[1] = 1.0 
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    # Wait 2 s and slew to second target
    await asyncio.sleep(2.)
    await mtcs.slew_icrs(ra=target_2.ra, dec=target_2.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Third target  # 1 um of z7 (y-coma)
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    wavefront_errors = np.zeros(19)
    wavefront_errors[3] = 1.0 
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    # Wait 2 s and slew to third target
    await asyncio.sleep(2.)
    await mtcs.slew_icrs(ra=target_3.ra, dec=target_3.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
    # Fourth target  # 1 um of z9 (trefoil)
    await mtcs.rem.mtaos.cmd_resetCorrection.start()
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    wavefront_errors = np.zeros(19)
    wavefront_errors[5] = 1.0 
    await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)
    await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

    # Wait 2 s and slew to fourth target
    await asyncio.sleep(2.)
    await mtcs.slew_icrs(ra=target_4.ra, dec=target_4.dec, rot_type=RotType.PhysicalSky)
    await asyncio.sleep(39.)
    
      
    # Stop tracking
    await mtcs.stop_tracking()
    
    t_end = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_end.format = "isot"
    print(f"Gathering data - with Aberrations - End time: {t_end.utc}")
    

## Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
# Query telemetry
df_aberr = await get_data_from_efd(
    exec_info.loc, 
    t_start, 
    t_end)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=4, sharex=True)

axs[0].plot(df_aberr["mount_el"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df_aberr["m1m3_z101"].dropna(), "C0o-", label="z")
axs[1].set_ylabel("M1M3 Forces\n[--]")

axs[2].plot(df_aberr["m2b1_applied"].dropna(), "C1^-", label="applied")
axs[2].plot(df_aberr["m2b1_gravLut"].dropna(), "C2v-", label="Gravity LUT")
axs[2].set_ylabel("M2 Forces\n[--]")

axs[3].plot(df_aberr["camhex_y"].dropna(), "C3x-", label="CamHex Y")
axs[3].plot(df_aberr["m2hex_y"].dropna(), "C4+-", label="M2Hex Y")
axs[3].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time_with_aberr.png")
plt.show()

## Compare against the corrections sent by MTAOS. 

check the force and hexapod position differences between steps 10 and 12.

Compare against the corrections sent by MTAOS. They should match.

### M1 forces vs MTAOS corrections

In [ ]:
m1m3_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m1m3Correction', 
    [f"zForces{i}" for i in range(156)], 
    start_time_aberr.utc, 
    end_time_aberr.utc
)

In [ ]:
m1m3_correction_applied = await client.select_time_series(
    'lsst.sal.MTM1M3.command_applyActiveOpticForces', 
    [f"zForces{i}" for i in range(156)], 
    start_time_aberr.utc, 
    end_time_aberr.utc
)

In [ ]:
plt.plot(m1m3_correction.T)
plt.plot(m1m3_correction_applied.T)

### M2 forces vs MTAOS corrections

In [ ]:
m2_correction = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2Correction', 
    [f"zForces{i}" for i in range(72)], 
    start_time_aberr.utc, 
    end_time_aberr.utc
)

In [ ]:
m2_correction_applied = await client.select_time_series(
    'lsst.sal.MTM2.command_applyForces', 
    [f"axial{i}" for i in range(72)], 
    start_time_aberr.utc, 
    end_time_aberr.utc

In [ ]:
plt.plot(m2_correction.T)
plt.plot(m2_correction_applied.T)

### CamHex vs MTAOS corrections

In [ ]:
cam_hexapod_correction_computed_xyz = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_cameraHexapodCorrection', 
    ["x", "y", "z"], 
    start.utc, 
    end.utc
)

cam_hexapod_correction_computed_uv = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_cameraHexapodCorrection', 
    ["u", "v"], 
    start.utc, 
    end.utc
)

In [ ]:
cam_hexapod_correction_applied_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["x", "y", "z", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=1
)

cam_hexapod_correction_applied_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["u", "v", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=1
)

In [ ]:
cam_hexapod_correction_command_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["x", "y", "z", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=1
)

cam_hexapod_correction_command_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["u", "v", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=1
)

### M2Hex vs MTAOS corrections

In [ ]:
m2_hexapod_correction_computed_xyz = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2HexapodCorrection', 
    ["x", "y", "z"], 
    start.utc, 
    end.utc
)

m2_hexapod_correction_computed_uv = await client.select_time_series(
    'lsst.sal.MTAOS.logevent_m2HexapodCorrection', 
    ["u", "v"], 
    start.utc, 
    end.utc
)

In [ ]:
m2_hexapod_correction_applied_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["x", "y", "z", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

m2_hexapod_correction_applied_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.logevent_uncompensatedPosition', 
    ["u", "v", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

In [ ]:
m2_hexapod_correction_command_xyz = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["x", "y", "z", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

m2_hexapod_correction_command_uv = await client.select_time_series(
    'lsst.sal.MTHexapod.command_move', 
    ["u", "v", "MTHexapodID"], 
    start.utc, 
    end.utc,
    index=2
)

## Wrap up
lower m1m3,
put hardware to OFFLINE
put all simulators to STANDBY
done.

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
await mtcs.point_azel(az=0, el=80)

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )